In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# Load the data
train_data = pd.read_csv('train.csv')

In [3]:
# Feature Engineering
train_data['living_to_total_area_ratio'] = train_data['life_sq'] / train_data['full_sq']
train_data['population_density'] = train_data['raion_popul'] / train_data['area_m']
train_data['green_to_industrial_ratio'] = train_data['green_zone_part'] / (train_data['indust_part'] + 0.0001)

In [4]:
# Replace infinite values with NaN
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
# Prepare the data
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']

In [6]:
# Identifying categorical and numerical columns
categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

In [7]:
# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

In [8]:
# Create the regression model
linear_model = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', LinearRegression())])

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Fit the model
linear_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['full_sq', 'life_sq',
                                                   'floor', 'area_m',
                                                   'raion_popul',
                                                   'green_zone_part',
                                                   'indust_part',
                                                   'children_preschool',
                                                   'preschool_education_centers_raion',
                                                   'children_school',
                                                   'school_education_centers_rai...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['product_type', 'sub_area',
                                                   'culture_objects_top_25',
                                                   'thermal_power_plant_raion',
                                                   'incineration_raion',
                                                   'oil_chemistry_raion',
                                                   'radiation_raion',
                                                   'railroad_terminal_raion',
                                                   'big_market_raion',
                                                   'nuclear_reactor_raion',
                                                   'detention_facility_raion',
                                                   'water_1line',
                                                   'big_road1_1line',
                                                   'railroad_1line',
                                                   'ecology'])])),
                ('regressor', LinearRegression())])

In [9]:
# Evaluate the model
predictions = linear_model.predict(X_val)
rmse = sqrt(mean_squared_error(y_val, predictions))
print(f'Validation RMSE: {rmse}')

Validation RMSE: 13815689.346568506


In [ ]:
test_data = pd.read_csv('test.csv')  

test_data['living_to_total_area_ratio'] = test_data['life_sq'] / test_data['full_sq']
test_data['population_density'] = test_data['raion_popul'] / test_data['area_m']
test_data['green_to_industrial_ratio'] = test_data['green_zone_part'] / (test_data['indust_part'] + 0.0001)

# Replace infinite values with NaN in the test dataset
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Predict using the trained model
test_predictions = linear_model.predict(test_data)

# Assuming 'Id' is the column in your test dataset that needs to be included in submission
submission = pd.DataFrame({
    'row ID': test_data['row ID'],
    'price_doc': test_predictions
})

# Save the submission file
submission.to_csv('multiple_linear_submission.csv', index=False)